# Wildlife Species Detection from Drone Imagery
## Data Exploration – WAID Dataset

**Project Objective:**  
This notebook explores the Wildlife Aerial Images from Drone (WAID) dataset.  
We analyze class distribution, image characteristics, and annotation quality to
ensure suitability for training deep learning–based object detection models.

**Dataset:** Wildlife Aerial Images from Drone (WAID)  
**Species:** Sheep, Cattle, Seal, Camelus, Kiang, Zebra


In [1]:
# Core libraries
import os
import json
import numpy as np
import pandas as pd

# Computer vision
import cv2
from PIL import Image

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Utilities
from collections import Counter
from tqdm import tqdm

# Plot settings
plt.style.use("seaborn-v0_8")
plt.rcParams["figure.figsize"] = (10, 6)
